# README

In questo file Jupyter si presentano le due euristiche da noi implementate per il problema del Fog Computing.

Per eseguire il codice bisogna usare le funzioni **run_heuristic** per la prima euristica, **multi_agente** per la seconda.

- a) **Parametri da passare alla prima euristica:**
     - 1) lista di richieste (che può variare ad ogni ciclo)
     - 2) lista di capacità (una volta fissata, resta costante per tutte le iterazioni)
     - 3) numero di nodi all'interno del sistema di fog computing
     - 4) un contatore di scambi

- b) **Parametri da passare alla seconda euristica:**
     - 1) lista della differenza tra richieste e capacità
     - 2) lista di richieste (che può variare ad ogni ciclo)
     - 3) lista di capacità (una volta fissata, resta costante per tutte le iterazioni)
     - 4) numero di nodi all'interno del sistema di fog computing
     - 5) un contatore di scambi
     
**NB**:
- 1) La lista di richieste deve essere estratta da una distribuzione di Poisson che abbia come media la media della capacità totale del sistema (capacità della cpu e capacità della ram). Il secondo paramentro è il numero di nodi.
- 2) La lista della capacità viene generata randomicamente in un range che va da 2 a 100 (2 è la richiesta di ram e cpu per ogni richiesta che viene fatta al sistema)

**NB 2**:
Il codice può essere eseguito anche senza iterazioni, ma, per una migliore simulazione, si consiglia di farle.

In [11]:
#vengono importate le librerie necessarie
import random
import numpy as np
import pandas as pd
import time

**Prima Euristica**: *i nodi hanno conoscenza dell'intero sistema*

Vengono di seguito definite le funzioni che serviranno poi per la costruzione della prima euristica *run_heuristic*

In [12]:
def diff(request,capacity,n):
    difference=[]
    for i in range(0,n):
        difference.append(capacity[i]-request[i])
    
    difference=sorted(difference,reverse=True)
        
    return difference

def iterate(difference,n,count):
    i=0
    while(difference[n-1]<0):
        difference[n-1]=difference[n-1] + difference[i]
        difference[i]=0
        i=i+1
        count+=1
    return difference,count


def fog_computing(difference,n,count):
    if(difference[n-1]>=0):
        print("No changing required:",difference)
    else:
        while(difference[n-1]<0):
            if(abs(difference[n-1]) <= abs(difference[0])):
                difference[0]=difference[0] + difference[n-1]
                difference[n-1]=0
                count+=1
            else:
                difference,count=iterate(difference,n,count)
            
            difference=sorted(difference,reverse=True)
        #print("System completed:",difference)
        print("Numero di scambi prima euristica:",count)
        return count

def run_heuristic(request,capacity,n,count):
    tot_request=sum(request)
    tot_capacity=sum(capacity)
    
    if(tot_request > tot_capacity):
        dif_tot=tot_request-tot_capacity
        i=0
        a=0
        while(dif_tot>0):
            request[i]=request[i]-1
            dif_tot=dif_tot-1
            i=i+1
            a=a+1
            if(i==n-1):
                i=0
        print("System overloaded. {} richieste rigettate".format(a))
        
        #print("Starting the process...")
        differenza=diff(request,capacity,n)
        scambi = fog_computing(differenza,n,count)
    else:
        #print("Starting the process...")
        differenza=diff(request,capacity,n)
        #print('Differenza_prima_euristica:',differenza)
        scambi = fog_computing(differenza,n,count)
    return scambi

**Seconda euristica**: *i nodi del sistema non hanno informazioni l'uno riguardo all'altro*

Vengono di seguito definite le funzioni che serviranno poi per la costruzione della seconda euristica *multi_agente*

In [59]:
def diff_multi_agente(request,capacity,n):
    difference=[]
    for i in range(0,n):
        difference.append(capacity[i]-request[i])
    return difference

def multi_agente(differenza, requests, n_richieste, m, scambi):
    indici=[]
    values=[]

    for i in range(len(differenza)):
        if(differenza[i]<0):
            indici.append(i)
            values.append(differenza[i])

    #print('Indici:',indici,'Valori:',values)  
    for i in indici:
        #print("Nodo:{}".format(i))        
        for j in range(len(differenza)):
            if(differenza[j]>=abs(values[indici.index(i)]) and j!=i):
                differenza[j]=differenza[j]+values[indici.index(i)]
                values[indici.index(i)]=0
                differenza[i]=0
                scambi+=1
                #print("Svuotamento completato")
                #print(differenza,values)
                break

        if(values[indici.index(i)]!=0):
            #print('Nodo {} da svuotare in splitting:'.format(i))
            #print(differenza,values)
            while(values[indici.index(i)] < 0):
                if(sum(differenza)>= 0): 
                    for k in range(len(differenza)):
                        if(differenza[k]>0):
                            if(abs(values[indici.index(i)])>differenza[k]):
                                values[indici.index(i)] = values[indici.index(i)] + differenza[k]
                                differenza[k]=0
                                scambi+=1
                            else:
                                differenza[k]=differenza[k]+values[indici.index(i)]
                                values[indici.index(i)]=0
                                scambi+=1
                            #print(differenza,values)
                else:
                    print('System overloaded')
                    if(abs(sum(differenza)) > abs(differenza[i])): 
                        print('{} richieste rigettate'.format(abs(differenza[i])))
                        values[indici.index(i)] = 0
                        differenza[i] = 0
                    else: 
                        print('{} richieste rigettate'.format(abs(sum(differenza))))
                        values[indici.index(i)] = values[indici.index(i)] - sum(differenza)
                        differenza[i] = differenza[i] - sum(differenza)          
            differenza[i]=0
            #print(differenza,values)
            #print('Nodo svuotato')
            
    print('Numero scambi seconda euristica:',scambi)
    return scambi

Di seguito vengono eseguite 10 iterazioni sia sulla prima che sulla seconda euristica, dove il numero di nodi viene settato a 6 (*m=6*), le richieste di ram e cpu vengono fissate a 2 (*ram_richiesta=2* e *cpu_richiesta=2*), le capacità di ram e cpu vengono settate randomicamente per gli m nodi, in un range che varia tra il numero di richiesta ram (o cpu) e 100.
Infine, il numero di richieste viene estratto da una distribuzione di Poisson, che ha come media la media della capacità totale del sistema (ovvero capacità della cpu e della ram).

In [14]:
for iter in range(0,10):
    print("ITERATION {}".format(iter+1))
    m=6
    ram_richiesta = 2
    cpu_richiesta = 2
    ram_capacity=random.sample(range(ram_richiesta,100),m)
    cpu_capacity=random.sample(range(cpu_richiesta,100),m)
    capacity_tot = []
    for i in range(0,m): 
        capacity_tot.append([ram_capacity[i],cpu_capacity[i]])
    n_richieste = [min(x) / ram_richiesta for x in capacity_tot]
    n_richieste = [int(x) for x in n_richieste]
    media_totale = np.mean(n_richieste).tolist()

    richieste=np.random.poisson(media_totale-0.5,m)

    differenza=diff_multi_agente(richieste,n_richieste,m)

    scambi=0
    count=0

    print("Richieste disponibili per nodo:",n_richieste)
    print("Richieste in entrata:",richieste)

    run_heuristic(richieste,n_richieste,m,count)

    multi_agente(differenza, richieste, n_richieste, m, scambi)

ITERATION 1
Richieste disponibili per nodo: [8, 1, 37, 11, 12, 10]
Richieste in entrata: [ 8 14 11 20 13 11]
Numero di scambi prima euristica: 4
Numero scambi seconda euristica: 4
ITERATION 2
Richieste disponibili per nodo: [35, 24, 14, 3, 33, 11]
Richieste in entrata: [12 23 17 22 24 18]
Numero di scambi prima euristica: 3
Numero scambi seconda euristica: 3
ITERATION 3
Richieste disponibili per nodo: [8, 3, 4, 9, 15, 9]
Richieste in entrata: [ 8  8  4  2 10  5]
Numero di scambi prima euristica: 1
Numero scambi seconda euristica: 1
ITERATION 4
Richieste disponibili per nodo: [3, 23, 1, 35, 26, 18]
Richieste in entrata: [17 22 15 20 22 15]
System overloaded. 5 richieste rigettate
Numero di scambi prima euristica: 5
System overloaded
5 richieste rigettate
Numero scambi seconda euristica: 5
ITERATION 5
Richieste disponibili per nodo: [17, 49, 12, 15, 23, 46]
Richieste in entrata: [31 32 22 32 24 30]
System overloaded. 9 richieste rigettate
Numero di scambi prima euristica: 4
System overlo

#### Applicazione su un caso reale: richieste di taxi per quartiere a Pechino

Di seguito, vengono applicate le euristiche implementate sopra ad un caso reale (un dataset riguardante le richieste di taxi in 18 quartieri diversi di Pechino). Il dataset è diviso in fasce orarie di mezz'ora. Nei chunks seguenti vengono creati dei dataframe divisi in giorni (*richieste_day_1*,*richieste_day_2*, ecc...).
I giorni vanno dal 2 all'8 febbraio 2008.
Ogni dataframe contiene tutte le fasce orarie di un singolo giorno (righe dei dataframe), per tutti i quartieri presi in considerazione (colonne dei dataframe).

In [16]:
df_originale=pd.read_csv("C:/Users/dasan/Desktop/csv_taxi.csv",sep=",")
df=pd.read_csv("C:/Users/dasan/Desktop/richieste_taxi.csv",sep=",")

In [114]:
media=(df_originale.groupby(["district"])['taxi_id'].mean()) + 850
district_capacity=round(media)
district_capacity = [int(x) for x in district_capacity]
district_capacity

[1514,
 12189,
 5654,
 2367,
 8378,
 2930,
 5946,
 5863,
 1511,
 1312,
 1892,
 3008,
 1924,
 3884,
 3620,
 3983,
 7488,
 1306]

In [115]:
richieste_day_1=df[0:21]
richieste_day_2=df[21:69].reset_index()
richieste_day_3=df[69:117].reset_index()
richieste_day_4=df[117:165].reset_index()
richieste_day_5=df[165:213].reset_index()
richieste_day_6=df[213:261].reset_index()
richieste_day_7=df[261:len(df)].reset_index()

In [116]:
quartieri=['quartiere_0','quartiere_1','quartiere_2','quartiere_3','quartiere_4','quartiere_5',
          'quartiere_6','quartiere_7','quartiere_8','quartiere_9','quartiere_10','quartiere_11',
           'quartiere_12','quartiere_13','quartiere_14','quartiere_15','quartiere_16','quartiere_17']

Di seguito vengono eseguite le due euristiche, per ogni giorno, su tutte le fasce orarie e su tutti i quartieri, per poi effettuare una successiva analisi dei risultati.

In [117]:
count=0
scambi=0
m=18
richieste=[]

for i in range(len(richieste_day_1)):
    for j in range(len(quartieri)):
        richieste.append(richieste_day_1['quartiere_{}'.format(j)][i])
    
    req=richieste.copy()
    
    print('ITERATION {}'.format(i))

    changes_1 = run_heuristic(richieste,district_capacity,m,count)
    
    differenza=diff_multi_agente(req,district_capacity,m)
    changes = multi_agente(differenza, req, district_capacity, m, scambi)
    time.sleep(10)
    richieste=[]

ITERATION 0
System overloaded. 2023 richieste rigettate
Numero di scambi prima euristica: 17
System overloaded
2023 richieste rigettate
Numero scambi seconda euristica: 44
ITERATION 1
Numero di scambi prima euristica: 16
Numero scambi seconda euristica: 34
ITERATION 2
Numero di scambi prima euristica: 15
Numero scambi seconda euristica: 33
ITERATION 3
Numero di scambi prima euristica: 11
Numero scambi seconda euristica: 39
ITERATION 4
Numero di scambi prima euristica: 11
Numero scambi seconda euristica: 39
ITERATION 5
System overloaded. 1556 richieste rigettate
Numero di scambi prima euristica: 17
System overloaded
1556 richieste rigettate
Numero scambi seconda euristica: 35
ITERATION 6
System overloaded. 187 richieste rigettate
Numero di scambi prima euristica: 17
System overloaded
187 richieste rigettate
Numero scambi seconda euristica: 33
ITERATION 7
Numero di scambi prima euristica: 15
Numero scambi seconda euristica: 33
ITERATION 8
System overloaded. 1623 richieste rigettate
Numer

In [118]:
count=0
scambi=0
m=18
richieste=[]

for i in range(len(richieste_day_2)):
    for j in range(len(quartieri)):
        richieste.append(richieste_day_2['quartiere_{}'.format(j)][i])
    
    req=richieste.copy()
    
    print('ITERATION {}'.format(i))

    changes_1 = run_heuristic(richieste,district_capacity,m,count)
    
    differenza=diff_multi_agente(req,district_capacity,m)
    changes = multi_agente(differenza, req, district_capacity, m, scambi)
    time.sleep(10)
    richieste=[]

ITERATION 0
Numero di scambi prima euristica: 4
Numero scambi seconda euristica: 4
ITERATION 1
Numero di scambi prima euristica: 3
Numero scambi seconda euristica: 3
ITERATION 2
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 3
No changing required: [2557, 1739, 1521, 1345, 1336, 1280, 1280, 1234, 1213, 1157, 1147, 1065, 968, 919, 890, 799, 627, 190]
Numero scambi seconda euristica: 0
ITERATION 4
No changing required: [7113, 4993, 4827, 3958, 3658, 3044, 2810, 2790, 2411, 2389, 2137, 1790, 1615, 1373, 1372, 1294, 1074, 1070]
Numero scambi seconda euristica: 0
ITERATION 5
Numero di scambi prima euristica: 1
Numero scambi seconda euristica: 1
ITERATION 6
No changing required: [3811, 2696, 2602, 2596, 2326, 2061, 1749, 1617, 1287, 1164, 1111, 1093, 952, 857, 701, 602, 528, 419]
Numero scambi seconda euristica: 0
ITERATION 7
No changing required: [4784, 4688, 3701, 3412, 3260, 2654, 2374, 2032, 1710, 1592, 1542, 1292, 1245, 1213, 1059, 946, 938, 789]
Numero

In [119]:
count=0
scambi=0
m=18
richieste=[]

for i in range(len(richieste_day_3)):
    for j in range(len(quartieri)):
        richieste.append(richieste_day_3['quartiere_{}'.format(j)][i])
    
    req=richieste.copy()
    
    print('ITERATION {}'.format(i))

    changes_1 = run_heuristic(richieste,district_capacity,m,count)
    
    differenza=diff_multi_agente(req,district_capacity,m)
    changes = multi_agente(differenza, req, district_capacity, m, scambi)
    time.sleep(10)
    richieste=[]

ITERATION 0
Numero di scambi prima euristica: 6
Numero scambi seconda euristica: 6
ITERATION 1
Numero di scambi prima euristica: 6
Numero scambi seconda euristica: 6
ITERATION 2
Numero di scambi prima euristica: 4
Numero scambi seconda euristica: 4
ITERATION 3
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 4
No changing required: [3301, 2651, 2219, 2130, 2013, 1866, 1750, 1668, 1520, 1295, 1280, 1159, 991, 956, 913, 727, 714, 621]
Numero scambi seconda euristica: 0
ITERATION 5
Numero di scambi prima euristica: 1
Numero scambi seconda euristica: 1
ITERATION 6
No changing required: [3341, 2558, 1894, 1868, 1708, 1591, 1536, 1359, 1295, 1168, 907, 828, 826, 731, 641, 602, 602, 270]
Numero scambi seconda euristica: 0
ITERATION 7
No changing required: [3785, 2622, 2508, 2483, 2213, 1656, 1446, 1383, 1380, 1289, 1029, 852, 829, 808, 733, 730, 562, 150]
Numero scambi seconda euristica: 0
ITERATION 8
No changing required: [4427, 3098, 2882, 2521, 2493, 2474, 2

In [120]:
count=0
scambi=0
m=18
richieste=[]

for i in range(len(richieste_day_4)):
    for j in range(len(quartieri)):
        richieste.append(richieste_day_4['quartiere_{}'.format(j)][i])
    
    req=richieste.copy()
    
    print('ITERATION {}'.format(i))

    changes_1 = run_heuristic(richieste,district_capacity,m,count)
    
    differenza=diff_multi_agente(req,district_capacity,m)
    changes = multi_agente(differenza, req, district_capacity, m, scambi)
    time.sleep(10)
    richieste=[]

ITERATION 0
Numero di scambi prima euristica: 4
Numero scambi seconda euristica: 4
ITERATION 1
Numero di scambi prima euristica: 1
Numero scambi seconda euristica: 1
ITERATION 2
Numero di scambi prima euristica: 1
Numero scambi seconda euristica: 1
ITERATION 3
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 4
No changing required: [10019, 7110, 6578, 5266, 5171, 4917, 3369, 3231, 3134, 2916, 2594, 2034, 1852, 1770, 1440, 1331, 1282, 1235]
Numero scambi seconda euristica: 0
ITERATION 5
No changing required: [4268, 3788, 3480, 2943, 2832, 2831, 2541, 1785, 1655, 1561, 1450, 1294, 1176, 1156, 1120, 981, 833, 636]
Numero scambi seconda euristica: 0
ITERATION 6
No changing required: [3719, 3231, 3190, 2791, 2576, 2415, 2332, 1630, 1525, 1288, 1210, 1148, 1121, 879, 797, 733, 650, 67]
Numero scambi seconda euristica: 0
ITERATION 7
No changing required: [3994, 3821, 3502, 2910, 2902, 2541, 2423, 1874, 1635, 1328, 1233, 1144, 1097, 951, 894, 883, 700, 512]
Nume

In [121]:
count=0
scambi=0
m=18
richieste=[]

for i in range(len(richieste_day_5)):
    for j in range(len(quartieri)):
        richieste.append(richieste_day_5['quartiere_{}'.format(j)][i])
    
    req=richieste.copy()
    
    print('ITERATION {}'.format(i))

    changes_1 = run_heuristic(richieste,district_capacity,m,count)
    
    differenza=diff_multi_agente(req,district_capacity,m)
    changes = multi_agente(differenza, req, district_capacity, m, scambi)
    time.sleep(10)
    richieste=[]

ITERATION 0
No changing required: [2238, 1677, 1511, 1243, 1107, 1027, 1023, 973, 892, 868, 855, 802, 637, 609, 572, 559, 315, 280]
Numero scambi seconda euristica: 0
ITERATION 1
No changing required: [3430, 3163, 2761, 2396, 2027, 1558, 1242, 1124, 1042, 951, 929, 903, 895, 689, 605, 601, 458, 345]
Numero scambi seconda euristica: 0
ITERATION 2
No changing required: [3582, 3333, 3234, 2496, 2133, 1780, 1255, 1157, 1094, 1016, 943, 943, 911, 893, 795, 618, 596, 278]
Numero scambi seconda euristica: 0
ITERATION 3
No changing required: [4078, 3784, 3466, 2861, 2651, 2582, 1214, 1047, 1030, 953, 944, 889, 834, 733, 703, 649, 475, 333]
Numero scambi seconda euristica: 0
ITERATION 4
No changing required: [5694, 4709, 4663, 3619, 3355, 2928, 1532, 1512, 1473, 1154, 1045, 1034, 917, 906, 874, 828, 671, 629]
Numero scambi seconda euristica: 0
ITERATION 5
No changing required: [4998, 4730, 4108, 3130, 2961, 2566, 1241, 1120, 1060, 955, 944, 885, 617, 609, 492, 488, 463, 239]
Numero scambi secon

In [122]:
count=0
scambi=0
m=18
richieste=[]

for i in range(len(richieste_day_6)):
    for j in range(len(quartieri)):
        richieste.append(richieste_day_6['quartiere_{}'.format(j)][i])
    
    req=richieste.copy()
    
    print('ITERATION {}'.format(i))

    changes_1 = run_heuristic(richieste,district_capacity,m,count)
    
    differenza=diff_multi_agente(req,district_capacity,m)
    changes = multi_agente(differenza, req, district_capacity, m, scambi)
    time.sleep(10)
    richieste=[]

ITERATION 0
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 1
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 2
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 3
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 4
No changing required: [8598, 6085, 5601, 4302, 4291, 3751, 2652, 2065, 1764, 1422, 1083, 1003, 996, 795, 699, 646, 486, 328]
Numero scambi seconda euristica: 0
ITERATION 5
Numero di scambi prima euristica: 3
Numero scambi seconda euristica: 3
ITERATION 6
Numero di scambi prima euristica: 1
Numero scambi seconda euristica: 1
ITERATION 7
Numero di scambi prima euristica: 3
Numero scambi seconda euristica: 3
ITERATION 8
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 9
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 10
Numero di scambi prima euristica: 2
Numero scambi seconda euristica:

In [123]:
count=0
scambi=0
m=18
richieste=[]

for i in range(len(richieste_day_7)):
    for j in range(len(quartieri)):
        richieste.append(richieste_day_7['quartiere_{}'.format(j)][i])
    
    req=richieste.copy()
    
    print('ITERATION {}'.format(i))

    changes_1 = run_heuristic(richieste,district_capacity,m,count)
    
    differenza=diff_multi_agente(req,district_capacity,m)
    changes = multi_agente(differenza, req, district_capacity, m, scambi)
    time.sleep(10)
    richieste=[]

ITERATION 0
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 1
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 2
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 3
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 4
No changing required: [10964, 7723, 6881, 5470, 5334, 5098, 3339, 3293, 3045, 2252, 2191, 1913, 1734, 1550, 1387, 1209, 1206, 1088]
Numero scambi seconda euristica: 0
ITERATION 5
No changing required: [9273, 6941, 6060, 5085, 4869, 4677, 2575, 2381, 2296, 1556, 1372, 1197, 1142, 1136, 1069, 1041, 860, 814]
Numero scambi seconda euristica: 0
ITERATION 6
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 7
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 8
Numero di scambi prima euristica: 2
Numero scambi seconda euristica: 2
ITERATION 9
Numero di scambi prima euristica: 3
Numero scambi se